In [62]:
using Revise
using Pkg
Pkg.activate("..")

"/home/kalmar/ownCloud/Julia-devel/ShapiroWilk/Project.toml"

In [63]:
using HypothesisTests
using StatsFuns
using Statistics

using Nemo

In [64]:
using Test
BF(s::S) where S<:AbstractString = BigFloat(replace(s, " "=>""))
@test BF(" 0.03404 06470 23559 61189 13744") == BigFloat("0.0340406470235596118913744")

Test Passed

In [65]:
includet("../src/normordstats.jl")
using Main.OrderStatistics

In [66]:
const CC = Nemo.AcbField(200)
@show normcdf(-CC("12.2"))

I(x, i, n) = normcdf(x)^(i-1)*normccdf(x)^(n-i)*normpdf(x)
let (n, i) = (20, 10), F = CC
    nf(k) = Nemo.gamma(F(k+1))
    A(n, i) = (nf(n)/nf(n-i))/nf(i-1)
    @time Nemo.integrate(F, x -> x*I(x, i, n), -12.0, 12.0)*A(n,i)
end

normcdf(-(CC("12.2"))) = [1.554119786389593509611458557357295129147864771630551057501e-34 +/- 8.89e-92] + i*0
  0.115766 seconds (108.45 k allocations: 6.881 MiB)


[-0.0619962864942923493110625602688838453864117810972437760308 +/- 6.42e-59] + i*0

In [71]:
@time OS20 = NormOrderStatistic(20, CC)

  0.539498 seconds (500.31 k allocations: 42.406 MiB, 1.73% gc time)


20-element NormOrderStatistic{acb}:
 [-1.867475059798320484736893084825229195653241537634846429962 +/- 5.54e-58] + i*0 
 [-1.407604095908403952123933422186710758669601197915440914782 +/- 3.68e-58] + i*0 
 [-1.130948052193125850730126682418451118896046258173387115667 +/- 5.38e-58] + i*0 
 [-0.920981700426101994048510948672504904667182411778452138237 +/- 5.36e-58] + i*0 
 [-0.745383005817130101336299361114331595157000841120411749431 +/- 5.63e-58] + i*0 
 [-0.59029692154291225317535565379084816212973830213362552334 +/- 4.24e-57] + i*0  
 [-0.448331753197445818665758169210881347655429748572537092012 +/- 4.56e-58] + i*0 
 [-0.314933241645695258449393096472683990196030031747981389083 +/- 5.54e-58] + i*0 
 [-0.1869573646823312249829624517298478826467862094535723895101 +/- 8.62e-59] + i*0
 [-0.0619962864942923493110625602688838453864117810972437760308 +/- 6.62e-59] + i*0
 [0.0619962864942923493110625602688838453864117810972437760308 +/- 6.62e-59] + i*0 
 [0.186957364682331224982962451729847882

In [72]:
OS10 = NormOrderStatistic(10, CC)
@time cov(OS10, 2, 3)
@time cov(OS10, 2, 3) - BF("0.14662 26179 78671 64928 38817")

# Old version (BigFloat)
# OS = NormOrderStatistic{BigFloat}(10)
# @time cov(OS, 2, 3)
# @time cov(OS, 2, 3) - BF("0.14662 26179 78671 64928 38817")
# 164.073075 seconds (48.11 M allocations: 3.283 GiB, 0.45% gc time)
# 173.991823 seconds (67.52 M allocations: 4.649 GiB, 0.63% gc time)
#   0.000083 seconds (158 allocations: 6.434 KiB)

# new version (Nemo):
#   8.880869 seconds (5.61 M allocations: 486.806 MiB, 1.61% gc time)
#  10.148706 seconds (8.21 M allocations: 619.306 MiB, 1.78% gc time)
#   0.024826 seconds (18.83 k allocations: 1001.310 KiB)

  0.000073 seconds (88 allocations: 4.844 KiB)
  0.000054 seconds (101 allocations: 5.590 KiB)


[-3.8790829461499616494399482235603e-26 +/- 6.93e-58] + i*0

In [69]:
using DelimitedFiles

function test_covariance(CC, N::Int; precision=1e-25)
    1 < N < 21 || throw("High precision covariance tables exist only for 2<N<21")
    OS = NormOrderStatistic(N, CC)
    covm = readdlm(joinpath("..", "data", "cov_$(lpad(N,2,'0')).csv"), ',')
    for k in 1:size(covm,1)
        n, i, j, s = view(covm, k, :)
        difference = abs(BF(s) - cov(OS, i, j))
        if difference > precision
            println("$i, $j \t ", difference)
        end
    end
end

function test_product(CC, N::Int; precision=1e-20)
    (N in (20, 30, 40, 50)) || throw("High precision product tables exist only for N=20,30,40,50")
    OS = NormOrderStatistic(N, CC)
    prdm = readdlm(joinpath("..", "data", "prd_$(lpad(N,2,'0')).csv"), ',')
    for k in 1:size(prdm,1)
        n, i, j, s =  view(prdm, k, :)
        difference = abs(BF(s) - expectation(OS, j, i))
        if difference > precision
            println("$i, $j \t ", difference)
        end
    end
end

test_product (generic function with 1 method)

In [150]:
test_covariance(CC, 10, precision=1e-25)
# 1, 12 	 [3.022905209474633167980380e-25 +/- 7.23e-50]
# 1, 13 	 [1.290594780008462136810957e-25 +/- 9.14e-50]
# 1, 14 	 [5.31121695764957957356299e-25 +/- 3.54e-49]
# 1, 15 	 [1.64256752056420132089112e-25 +/- 5.50e-49]
# 1, 16 	 [3.09857426514853100793707e-25 +/- 4.74e-49]
# 2, 13 	 [1.18008765594399490895934e-25 +/- 3.91e-49]
# 2, 14 	 [1.15480455983800801280229e-25 +/- 6.68e-49]
# 2, 15 	 [1.70744921057212630710129e-25 +/- 6.44e-49]
# 2, 16 	 [1.08814654230047908728308e-25 +/- 7.05e-49]
# 2, 17 	 [1.39331540831508921067100e-25 +/- 3.67e-49]
# 3, 16 	 [1.1151309925136943285743e-25 +/- 1.37e-48]

┌ Info: ψ
│   i = 8
│   j = 2
└ @ Main.OrderStatistics /home/kalmar/ownCloud/Julia-devel/ShapiroWilk/src/normordstats.jl:178


  8.386625 seconds (3.84 M allocations: 356.517 MiB, 1.28% gc time)


┌ Info: ψ
│   i = 7
│   j = 2
└ @ Main.OrderStatistics /home/kalmar/ownCloud/Julia-devel/ShapiroWilk/src/normordstats.jl:178


  8.457481 seconds (3.69 M allocations: 348.869 MiB, 1.27% gc time)


┌ Info: ψ
│   i = 8
│   j = 2
└ @ Main.OrderStatistics /home/kalmar/ownCloud/Julia-devel/ShapiroWilk/src/normordstats.jl:178


...

 10.694864 seconds (4.85 M allocations: 448.901 MiB, 1.44% gc time)


┌ Info: ψ
│   i = 4
│   j = 3
└ @ Main.OrderStatistics /home/kalmar/ownCloud/Julia-devel/ShapiroWilk/src/normordstats.jl:178


  8.511889 seconds (3.82 M allocations: 356.666 MiB, 1.32% gc time)


┌ Info: ψ
│   i = 5
│   j = 3
└ @ Main.OrderStatistics /home/kalmar/ownCloud/Julia-devel/ShapiroWilk/src/normordstats.jl:178


InterruptException: InterruptException:

In [162]:
OrderStatistics.dropcache()

In [174]:
Set([CC(1), CC(2) - 1])

Set(acb[1.000000000000000000000000000000000000000000000000000000000000 + i*0])

In [167]:
Base.hash(::acb, h::UInt) = h

In [168]:
hash(CC(2) - 1)

0x0000000000000000

In [169]:
OrderStatistics.ψ(CC, 2, 3, OrderStatistics.R)

┌ Info: ψ
│   i = 3
│   j = 2
└ @ Main.OrderStatistics /home/kalmar/ownCloud/Julia-devel/ShapiroWilk/src/normordstats.jl:178


  4.709096 seconds (2.43 M allocations: 221.766 MiB, 1.78% gc time)


[0.02132627534580291857508603603199104378779996721174769085502 +/- 8.53e-60] + i*0

In [170]:
OrderStatistics.ψ(CC, 3, 2, OrderStatistics.R)

[0.02132627534580291857508603603199104378779996721174769085502 +/- 8.53e-60] + i*0

In [161]:
OrderStatistics._cache[Symbol("Main.OrderStatistics.ψ_int")][acb]

Dict{Tuple{Int64,Int64,acb},acb} with 4 entries:
  (3, 2, 12.19999999999999… => [0.021326275345802918575086036031991043787799967…
  (3, 2, 12.19999999999999… => [0.021326275345802918575086036031991043787799967…
  (3, 2, 12.00000000000000… => [0.021326275345802918575086036031991043787799967…
  (3, 2, 12.00000000000000… => [0.021326275345802918575086036031991043787799967…

In [171]:
OrderStatistics.ψ(CC, 3, 2, 12)

┌ Info: ψ
│   i = 3
│   j = 2
└ @ Main.OrderStatistics /home/kalmar/ownCloud/Julia-devel/ShapiroWilk/src/normordstats.jl:178


  4.607376 seconds (2.40 M allocations: 219.001 MiB, 1.55% gc time)


[0.02132627534580291857508603603199104378779996721174769085502 +/- 8.41e-60] + i*0

In [172]:
OrderStatistics.ψ(CC, 2, 3, 12)

[0.02132627534580291857508603603199104378779996721174769085502 +/- 8.41e-60] + i*0

In [108]:
test_covariance(CC, 20, precision=1e-25)

[0.999999999999999999999999999999953119357157583940944627552 +/- 1.79e-58] + i*0

In [88]:
OrderStatistics.getval!(OrderStatistics.β, acb, (CC, 0, 0, 16.6)...)

[1.000000000000000000000000000000000000000000000000000000000 +/- 9.35e-59] + i*0

In [55]:
OS = NormOrderStatistic(20, CC)

20-element NormOrderStatistic{acb}:
 [-1.867475059798320484736893084825229195653241537634846429962 +/- 5.54e-58] + i*0 
 [-1.407604095908403952123933422186710758669601197915440914782 +/- 3.68e-58] + i*0 
 [-1.130948052193125850730126682418451118896046258173387115667 +/- 5.38e-58] + i*0 
 [-0.920981700426101994048510948672504904667182411778452138237 +/- 5.36e-58] + i*0 
 [-0.745383005817130101336299361114331595157000841120411749431 +/- 5.63e-58] + i*0 
 [-0.59029692154291225317535565379084816212973830213362552334 +/- 4.24e-57] + i*0  
 [-0.448331753197445818665758169210881347655429748572537092012 +/- 4.56e-58] + i*0 
 [-0.314933241645695258449393096472683990196030031747981389083 +/- 5.54e-58] + i*0 
 [-0.1869573646823312249829624517298478826467862094535723895101 +/- 8.62e-59] + i*0
 [-0.0619962864942923493110625602688838453864117810972437760308 +/- 6.62e-59] + i*0
 [0.0619962864942923493110625602688838453864117810972437760308 +/- 6.62e-59] + i*0 
 [0.186957364682331224982962451729847882

In [93]:
OrderStatistics.setradius!(16.6)

Radius of integration is set to 16.6.

In [94]:
OS

10-element NormOrderStatistic{acb}:
 [-1.538752730835172856027531899751416692373952366627682103599 +/- 3.05e-58] + i*0 
 [-1.001357044575814358534364083601898843035220230488969304480 +/- 2.98e-58] + i*0 
 [-0.656059105364761203258548158914697623470120887507987807050 +/- 2.64e-58] + i*0 
 [-0.375764696997877539385948987052625083232194837704774404915 +/- 4.37e-58] + i*0 
 [-0.1226677522843380642246518815271890894008591432659831248085 +/- 6.99e-59] + i*0
 [0.1226677522843380642246518815271890894008591432659831248085 +/- 6.99e-59] + i*0 
 [0.375764696997877539385948987052625083232194837704774404915 +/- 4.37e-58] + i*0  
 [0.656059105364761203258548158914697623470120887507987807050 +/- 2.64e-58] + i*0  
 [1.001357044575814358534364083601898843035220230488969304480 +/- 2.98e-58] + i*0  
 [1.538752730835172856027531899751416692373952366627682103599 +/- 3.05e-58] + i*0  

In [137]:
@time @testset "Relations between α, β and expectations/moments of OS" begin
    OS = NormOrderStatistic(50, CC)
    (α, β) = (OrderStatistics.α_cached, OrderStatistics.β_cached)
    eps = 4e-26
    R = OrderStatistics.RADIUS.R
    n = OS.n
    @testset "β_ii" begin
        for i in 1:n
            beta_ii_residual = β(CC, i, i, R) - ( i*β(CC, i-1, i-1, R)/(4i + 2) -  2α(CC, i+1, i, R)/(2i + 1))
            check, int = unique_integer(beta_ii_residual)
            @test check && int == 0
            @test midpoint(real(beta_ii_residual)) < eps
        end
    end
    
    @testset "α_ij" begin
        for i in 1:n
            for j in 1:n
                alpha_ij_residual = α(CC, i, j, R) - ( α(CC, i, j+1, R) + α(CC, i+1, j, R) )
                check, int = unique_integer(alpha_ij_residual)
                @test check && int == 0
                @test midpoint(real(alpha_ij_residual)) < eps
            end
        end
    end

    @testset "β_ij" begin
        for i in 1:n
            for j in 1:n
                beta_ij_residual = β(CC, i, j, R) - ( β(CC, i, j+1, R) + β(CC, i+1, j, R) )
                check, int = unique_integer(beta_ij_residual)
                @test check && int == 0
                @test midpoint(real(beta_ij_residual)) < eps
            end
        end
    end
    
    @testset "sum of products and moments" begin
        
        eps = 10^(OS.n/2)*eps
        
        for i in 1:n-1
            res = sum(expectation(OS, i, j) for j in 1:OS.n)
            check, int = unique_integer(res)
            @test check && int == 1
            @test real(res) - int < eps
        end
        
        res = sum(moment(OS, i, pow=2) for i in 1:OS.n)
        check, int = unique_integer(res)
        @test check && int == OS.n
        @test real(res) - int < eps
    end
    

end;         

Test Summary:                                         |  Pass  Total
Relations between α, β and expectations/moments of OS | 10200  10200
 20.942519 seconds (42.08 M allocations: 2.054 GiB, 4.26% gc time)


In [138]:
test_product(CC, 20, precision=1e-25)
# 1, 12 	 [2.397925841562081652369698e-25 +/- 6.27e-50]
# 1, 15 	 [1.128850023517018841484787e-24 +/- 3.13e-49]
# 1, 16 	 [1.561386680199541941654516e-24 +/- 6.49e-49]
# 1, 17 	 [1.477051332814374888883390e-24 +/- 5.20e-49]
# 1, 18 	 [7.16515824283871448812541e-25 +/- 5.46e-49]
# 1, 19 	 [2.009504451860344687539499e-25 +/- 4.43e-50]
# 2, 13 	 [1.98170958935095581676613e-25 +/- 2.31e-49]
# 2, 14 	 [2.726211060871447556016067e-24 +/- 5.98e-49]
# 2, 15 	 [8.311930867749897816088249e-24 +/- 2.81e-49]
# 2, 16 	 [1.3184138551373367660245296e-23 +/- 7.96e-49]
# 2, 17 	 [1.1671410815755033763276798e-23 +/- 7.98e-49]
# 2, 18 	 [5.998079817564746558457810e-24 +/- 4.96e-49]
# 2, 19 	 [1.594125465319563702640011e-24 +/- 3.03e-49]
# 3, 12 	 [1.644135161591164306250331e-25 +/- 5.73e-50]
# 3, 13 	 [5.74520215560577522441012e-25 +/- 2.59e-49]
# 3, 14 	 [7.534083313743807009900801e-24 +/- 1.95e-49]
# 3, 15 	 [2.4683689703891136477993711e-23 +/- 4.02e-49]
# 3, 16 	 [4.0612596289679394769904232e-23 +/- 6.39e-49]
# 3, 17 	 [3.8124034787151244991254162e-23 +/- 7.56e-49]
# 3, 18 	 [2.0624096764358486690426377e-23 +/- 7.94e-49]
# 4, 12 	 [2.492270949935806850395806e-25 +/- 4.59e-50]
# 4, 13 	 [4.80231602129821092389729e-25 +/- 1.19e-49]
# 4, 14 	 [1.0780574059339791025566921e-23 +/- 3.12e-49]
# 4, 15 	 [3.8769748907962868683055622e-23 +/- 6.42e-49]
# 4, 16 	 [6.741136245049678696976625e-23 +/- 4.16e-48]
# 4, 17 	 [6.662730097250939140496623e-23 +/- 1.53e-48]
# 5, 12 	 [3.594688675671857020108529e-25 +/- 3.53e-50]
# 5, 13 	 [3.270989461863411576029765e-25 +/- 6.98e-50]
# 5, 14 	 [8.037586965898718717185743e-24 +/- 2.65e-49]
# 5, 15 	 [3.4303273654712703928286185e-23 +/- 2.09e-49]
# 5, 16 	 [6.4584358138956485138644892e-23 +/- 4.51e-49]
# 6, 12 	 [3.7402452830295599052835698e-25 +/- 4.76e-51]
# 6, 13 	 [1.0364820890476395478831981e-24 +/- 5.22e-50]
# 6, 14 	 [2.0991688659917754259138108e-24 +/- 5.27e-50]
# 6, 15 	 [1.5813525429147822948086553e-23 +/- 2.26e-49]
# 7, 12 	 [1.6753402970516184071856219e-25 +/- 2.75e-51]
# 7, 13 	 [9.1724779080163595250951863e-25 +/- 4.69e-51]
# 7, 14 	 [9.919111991127246886208039e-25 +/- 3.94e-50]
# 8, 12 	 [1.20800694314989709804806837e-25 +/- 2.33e-52]
# 8, 13 	 [3.8026938294729510008091417e-25 +/- 3.91e-51]

1, 12 	 [2.397925841562081652369698e-25 +/- 6.27e-50]
1, 15 	 [1.128850023517018841484787e-24 +/- 3.13e-49]
1, 16 	 [1.561386680199541941654516e-24 +/- 6.49e-49]
1, 17 	 [1.477051332814374888883390e-24 +/- 5.20e-49]
1, 18 	 [7.16515824283871448812541e-25 +/- 5.46e-49]
1, 19 	 [2.009504451860344687539499e-25 +/- 4.43e-50]
2, 13 	 [1.98170958935095581676613e-25 +/- 2.31e-49]
2, 14 	 [2.726211060871447556016067e-24 +/- 5.98e-49]
2, 15 	 [8.311930867749897816088249e-24 +/- 2.81e-49]
2, 16 	 [1.3184138551373367660245296e-23 +/- 7.96e-49]
2, 17 	 [1.1671410815755033763276798e-23 +/- 7.98e-49]
2, 18 	 [5.998079817564746558457810e-24 +/- 4.96e-49]
2, 19 	 [1.594125465319563702640011e-24 +/- 3.03e-49]
3, 12 	 [1.644135161591164306250331e-25 +/- 5.73e-50]
3, 13 	 [5.74520215560577522441012e-25 +/- 2.59e-49]
3, 14 	 [7.534083313743807009900801e-24 +/- 1.95e-49]
3, 15 	 [2.4683689703891136477993711e-23 +/- 4.02e-49]
3, 16 	 [4.0612596289679394769904232e-23 +/- 6.39e-49]
3, 17 	 [3.8124034787151244

In [144]:
CC("1.1")

[1.10000000000000000000000000000000000000000000000000000000000 +/- 2.25e-60] + i*0

In [145]:
normcdf(-18.0)

9.740948918936769e-73

In [142]:
test_product(CC, 40, precision=1e-15)

1, 25 	 [1.64812462225345995143641e-14 +/- 1.03e-38]
1, 26 	 [9.04246058569913603036822e-14 +/- 1.68e-38]
1, 27 	 [2.294655016608422126483138e-13 +/- 5.15e-38]
1, 28 	 [1.103339158496451140170722e-13 +/- 3.17e-38]
1, 29 	 [1.1110895884151236301120250e-12 +/- 6.59e-38]
1, 30 	 [4.0748597150120070531205099e-12 +/- 8.01e-38]
1, 31 	 [7.8544041161591486735876709e-12 +/- 7.87e-38]
1, 32 	 [1.00179898989046402947245624e-11 +/- 8.87e-38]
1, 33 	 [8.9133591768197916489352340e-12 +/- 7.87e-38]
1, 34 	 [5.5070865103671287827487695e-12 +/- 3.08e-38]
1, 35 	 [2.2075631225342109863945535e-12 +/- 2.03e-38]
1, 36 	 [4.3017495717354880744052559e-13 +/- 6.01e-39]
1, 37 	 [6.344944250372154821601947e-14 +/- 3.02e-39]
1, 38 	 [6.4392436174529015274541922e-14 +/- 5.88e-40]
1, 39 	 [1.6402423778866321389241463e-14 +/- 4.48e-40]
1, 40 	 [1.5709303168667301654640807e-15 +/- 2.96e-41]
2, 23 	 [1.5659800324741694264468e-15 +/- 4.46e-38]
2, 24 	 [1.98087854522506949713988e-14 +/- 3.88e-38]
2, 25 	 [2.6190973509

13, 21 	 [3.09258946182675069903887557e-14 +/- 7.33e-41]
13, 22 	 [5.6686943345422652420636711e-14 +/- 9.06e-40]
13, 23 	 [2.75188627916975890716812911e-12 +/- 3.87e-39]
13, 24 	 [2.65436907455430272168547989e-11 +/- 4.45e-38]
13, 25 	 [1.340314254376552016510353076e-10 +/- 6.47e-38]
13, 26 	 [4.86400544872677830033828086e-10 +/- 4.50e-37]
13, 27 	 [1.43391128268065061018992529e-9 +/- 4.06e-36]
13, 28 	 [3.54545851095047104092291642e-9 +/- 6.36e-36]
14, 21 	 [1.69953866029452222837587894e-14 +/- 4.97e-41]
14, 22 	 [8.2011542113251386210210308e-14 +/- 5.64e-40]
14, 23 	 [9.19733067546223418362913879e-13 +/- 6.86e-40]
14, 24 	 [1.160590141994088604263341695e-11 +/- 7.08e-39]
14, 25 	 [6.31461046528905471573150540e-11 +/- 5.82e-38]
14, 26 	 [2.27979758804004667629877338e-10 +/- 1.55e-37]
14, 27 	 [6.31172467187126225692782862e-10 +/- 2.65e-37]
15, 21 	 [7.56853506669232592343432635e-15 +/- 2.96e-42]
15, 22 	 [5.68277738926894212068284449e-14 +/- 1.92e-41]
15, 23 	 [1.258065671010797581494

In [39]:
expectationproduct(NormOrderStatistic(30, CC), 10, 14)

[0.099929542470794630696226575408301854857877276916 +/- 4.96e-49] + i*0

In [41]:
expectationproduct(NormOrderStatistic(40, CC), 2, 32)

[-1.380905890052630484691883043699963541 +/- 6.20e-37] + i*0

In [43]:
test_product(CC, 40, precision=1e-15)

1, 25 	 [1.648124622253460974539092e-14 +/- 9.41e-39]
1, 26 	 [9.04246058569913500730451e-14 +/- 1.34e-38]
1, 27 	 [2.294655016608422228785525e-13 +/- 3.45e-38]
1, 28 	 [1.103339158496451037872429e-13 +/- 3.71e-38]
1, 29 	 [1.1110895884151236198826190e-12 +/- 5.33e-38]
1, 30 	 [4.0748597150120070633494755e-12 +/- 4.50e-38]
1, 31 	 [7.8544041161591486633591670e-12 +/- 3.20e-38]
1, 32 	 [1.00179898989046403049525777e-11 +/- 6.95e-38]
1, 33 	 [8.9133591768197916387077413e-12 +/- 2.35e-38]
1, 34 	 [5.5070865103671287929756957e-12 +/- 2.83e-38]
1, 35 	 [2.2075631225342109761682524e-12 +/- 2.10e-38]
1, 36 	 [4.3017495717354881766612042e-13 +/- 4.29e-39]
1, 37 	 [6.344944250372155844078858e-14 +/- 1.44e-39]
1, 38 	 [6.4392436174529005050791581e-14 +/- 6.20e-40]
1, 39 	 [1.64024237788663316116055455e-14 +/- 5.79e-41]
1, 40 	 [1.57093031686671995805490404e-15 +/- 4.63e-42]
2, 23 	 [1.5659800324741694264468e-15 +/- 4.31e-38]
2, 24 	 [1.98087854522506949713988e-14 +/- 3.28e-38]
2, 25 	 [2.6190973

11, 28 	 [1.1969975471150948632144796976e-8 +/- 6.69e-37]
11, 29 	 [2.524887047376416362928825834e-8 +/- 1.42e-36]
11, 30 	 [4.026849729388514208488727382e-8 +/- 4.66e-36]
12, 21 	 [4.592029500089457197282812903e-14 +/- 7.46e-42]
12, 22 	 [6.05610976009643360154518742e-14 +/- 2.92e-41]
12, 23 	 [5.497176363487583647399554079e-12 +/- 2.03e-40]
12, 24 	 [4.554684432610194127869346417e-11 +/- 3.42e-39]
12, 25 	 [2.2303854290566429511124109736e-10 +/- 8.90e-39]
12, 26 	 [8.501236268531311224826225038e-10 +/- 5.03e-38]
12, 27 	 [2.730903831501745753708898687e-9 +/- 4.84e-37]
12, 28 	 [7.246468190558043503112080117e-9 +/- 2.94e-37]
12, 29 	 [1.5312058715179628166798747734e-8 +/- 9.37e-37]
13, 21 	 [3.092589461826750699038875572e-14 +/- 5.11e-42]
13, 22 	 [5.66869433454226524206367115e-14 +/- 5.16e-41]
13, 23 	 [2.7518862791697589071681291110e-12 +/- 8.26e-41]
13, 24 	 [2.6543690745543027216854798870e-11 +/- 3.76e-40]
13, 25 	 [1.3403142543765520165103530760e-10 +/- 5.69e-39]
13, 26 	 [4.8640

In [134]:
test_product(CC, 50, precision=1e-10)
# Info: ψ
# │   i = 1
# │   j = 49
# └ @ HypothesisTests.OrderStatistics /home/kalmar/.julia/dev/HypothesisTests/src/normordstats.jl:160

#  22.713496 seconds (18.05 M allocations: 1.480 GiB, 2.70% gc time)

# ┌ Info: ψ
# │   i = 1
# │   j = 48
# └ @ HypothesisTests.OrderStatistics /home/kalmar/.julia/dev/HypothesisTests/src/normordstats.jl:160

#  23.369847 seconds (18.73 M allocations: 1.509 GiB, 2.68% gc time)

# ┌ Info: ψ
# │   i = 2
# │   j = 48
# └ @ HypothesisTests.OrderStatistics /home/kalmar/.julia/dev/HypothesisTests/src/normordstats.jl:160

#  20.472566 seconds (16.49 M allocations: 1.328 GiB, 2.72% gc time)


1, 29 	 [3.0970791791836003212680e-10 +/- 5.74e-33]
1, 30 	 [9.1937977849363155082912e-10 +/- 7.08e-33]
1, 31 	 [3.2288866783879601899396e-9 +/- 4.91e-32]
1, 32 	 [3.48830091826816985422608e-8 +/- 4.40e-32]
1, 33 	 [1.143379334139772491482727e-7 +/- 8.31e-32]
1, 34 	 [1.503025851417404123552178e-7 +/- 7.00e-32]
1, 35 	 [1.88229370176690755093080e-7 +/- 5.46e-31]
1, 36 	 [1.278484600625643554451401e-6 +/- 2.51e-31]
1, 37 	 [2.925153687045560375029120e-6 +/- 4.33e-31]
1, 38 	 [4.022566130674906884648024e-6 +/- 2.91e-31]
1, 39 	 [3.379262961776634428963045e-6 +/- 2.61e-31]
1, 40 	 [1.083701716388943676982522e-6 +/- 2.21e-31]
1, 41 	 [1.356163881296240596374701e-6 +/- 5.77e-31]
1, 42 	 [2.476252300245090123566134e-6 +/- 3.16e-31]
1, 43 	 [2.1297178580715769963678048e-6 +/- 8.56e-32]
1, 44 	 [1.1899152934839677351253629e-6 +/- 4.12e-32]
1, 45 	 [4.504544718084153459289746e-7 +/- 3.00e-32]
1, 46 	 [1.1033080612567918977462652e-7 +/- 4.32e-33]
1, 47 	 [1.436227739182699175273300e-8 +/- 4.63e-

8, 40 	 [0.39946465095928701344842077 +/- 4.11e-27]
8, 41 	 [0.40415296370445590165103110 +/- 5.14e-27]
8, 42 	 [0.290292573743804827566749444 +/- 8.63e-28]
8, 43 	 [0.150792890005047055808046880 +/- 9.19e-28]
9, 26 	 [9.9908672510823984762402e-9 +/- 4.03e-32]
9, 27 	 [1.0340535984407170758348983e-6 +/- 6.44e-32]
9, 28 	 [5.087954722911891361562903e-6 +/- 2.79e-31]
9, 29 	 [5.908533530056718216921023e-5 +/- 2.95e-30]
9, 30 	 [0.00067745525882896941890112192 +/- 3.55e-30]
9, 31 	 [0.00285636219182735786646690457 +/- 6.07e-30]
9, 32 	 [0.0049416268199013172007942339 +/- 2.71e-29]
9, 33 	 [0.0065798488294160340841369010 +/- 4.00e-29]
9, 34 	 [0.0616327912083332307599061702 +/- 8.35e-29]
9, 35 	 [0.180446492864240662588819323 +/- 3.90e-28]
9, 36 	 [0.318232762278380586220859085 +/- 4.30e-28]
9, 37 	 [0.351354497656372311141495854 +/- 4.13e-28]
9, 38 	 [0.173184784559576896493477138 +/- 6.87e-28]
9, 39 	 [0.172046695367150859127653607 +/- 6.10e-28]
9, 40 	 [0.48920455389954970388419648 +/- 

22, 27 	 [3.08690403524655469282197909168e-10 +/- 1.64e-40]
22, 28 	 [2.47389928383182597804966481004e-9 +/- 3.39e-39]
22, 29 	 [5.68766823455443748256693638649e-8 +/- 5.16e-38]
23, 28 	 [2.60639285800835808819079686309e-10 +/- 2.28e-40]


In [41]:
OrderStatistics.dropcache()

In [42]:
@time OS20_new = NormOrderStatistic{BigFloat}(20)

  5.756520 seconds (1.35 M allocations: 71.232 MiB, 0.31% gc time)


20-element NormOrderStatistic{BigFloat}:
 -1.867475059798320484736893084824838090888801661746456625680223450741376647578799  
 -1.40760409590840395212393342218671075866960119791544091478225942860701901186091   
 -1.130948052193125850730126682418451118896046258173387115667448774243390711988079  
 -0.920981700426101994048510948672504904667182411778452138236644019346596766961803  
 -0.7453830058171301013362993611143315951570008411204117494314936273795438793145304 
 -0.5902969215429122531753556537908481621297383021336255233431990175176868086957988 
 -0.4483317531974458186657581692108813476554297485725370920116536339830903486746654 
 -0.3149332416456952584493930964726839901960300317479813890825947751038509796136689 
 -0.186957364682331224982962451729847882646786209453572389510154765420258047122795  
 -0.06199628649429234931106256026888384538641178109724377603083994149747006035083674
  0.06199628649429234931106256026888384538641178109724377603083994149747006035083674
  0.18695736468233122498

In [26]:
@time OS20 = NormOrderStatistic{BigFloat}(20)

20-element NormOrderStatistic{BigFloat}:
 -1.867475059798320484736893084824838090888801661746456625680223450741376647578799  
 -1.40760409590840395212393342218671075866960119791544091478225942860701901186091   
 -1.130948052193125850730126682418451118896046258173387115667448774243390711988079  
 -0.920981700426101994048510948672504904667182411778452138236644019346596766961803  
 -0.7453830058171301013362993611143315951570008411204117494314936273795438793145304 
 -0.5902969215429122531753556537908481621297383021336255233431990175176868086957988 
 -0.4483317531974458186657581692108813476554297485725370920116536339830903486746654 
 -0.3149332416456952584493930964726839901960300317479813890825947751038509796136689 
 -0.186957364682331224982962451729847882646786209453572389510154765420258047122795  
 -0.06199628649429234931106256026888384538641178109724377603083994149747006035083674
  0.06199628649429234931106256026888384538641178109724377603083994149747006035083674
  0.18695736468233122498

In [26]:
test_covariance(BigFloat, 20, precision=1e-25) #with logarithms

165.370075 seconds (34.26 M allocations: 1.691 GiB, 0.33% gc time)
164.691264 seconds (33.88 M allocations: 1.673 GiB, 0.32% gc time)
165.670620 seconds (33.88 M allocations: 1.673 GiB, 0.32% gc time)
170.622013 seconds (33.60 M allocations: 1.659 GiB, 0.32% gc time)
163.295209 seconds (33.60 M allocations: 1.659 GiB, 0.33% gc time)
162.592262 seconds (33.60 M allocations: 1.659 GiB, 0.33% gc time)
159.729990 seconds (33.03 M allocations: 1.631 GiB, 0.33% gc time)
177.545984 seconds (33.03 M allocations: 1.631 GiB, 0.32% gc time)
167.067635 seconds (33.03 M allocations: 1.631 GiB, 0.33% gc time)
163.788128 seconds (33.03 M allocations: 1.631 GiB, 0.34% gc time)
159.186637 seconds (32.10 M allocations: 1.585 GiB, 0.34% gc time)
158.115396 seconds (32.69 M allocations: 1.614 GiB, 0.34% gc time)
156.894415 seconds (32.69 M allocations: 1.614 GiB, 0.33% gc time)
155.864343 seconds (32.69 M allocations: 1.614 GiB, 0.35% gc time)
156.834767 seconds (32.69 M allocations: 1.614 GiB, 0.34% gc t

114.466423 seconds (21.93 M allocations: 1.086 GiB, 0.86% gc time)
114.743330 seconds (21.93 M allocations: 1.086 GiB, 0.87% gc time)
114.411670 seconds (21.93 M allocations: 1.086 GiB, 0.87% gc time)
1, 16 	 3.098574265148531007937066415560053624742867674639568108228820856274820301763986e-25
 69.690639 seconds (13.12 M allocations: 664.843 MiB, 0.88% gc time)
 69.951304 seconds (13.12 M allocations: 664.842 MiB, 0.83% gc time)
105.993168 seconds (20.28 M allocations: 1.004 GiB, 0.86% gc time)
105.483583 seconds (20.28 M allocations: 1.004 GiB, 0.69% gc time)
105.656516 seconds (20.28 M allocations: 1.004 GiB, 0.69% gc time)
108.237562 seconds (20.70 M allocations: 1.025 GiB, 0.67% gc time)
107.790910 seconds (20.70 M allocations: 1.025 GiB, 0.67% gc time)
107.938459 seconds (20.70 M allocations: 1.025 GiB, 0.66% gc time)
107.834650 seconds (20.70 M allocations: 1.025 GiB, 0.67% gc time)
107.683952 seconds (20.70 M allocations: 1.025 GiB, 0.67% gc time)
107.671534 seconds (20.70 M allo

In [28]:
test_product(BigFloat, 30, precision=1e-20) #with logarithms

187.777815 seconds (39.10 M allocations: 1.929 GiB, 0.74% gc time)
183

.461981 seconds (38.23 M allocations: 1.887 GiB, 0.73% gc time)
182.563965 seconds (38.23 M allocations: 1.887 GiB, 0.73% gc time)
188.264331 seconds (38.23 M allocations: 1.887 GiB, 0.72% gc time)
182.823237 seconds (36.97 M allocations: 1.825 GiB, 0.72% gc time)
188.334023 seconds (36.97 M allocations: 1.825 GiB, 0.74% gc time)
182.971128 seconds (36.97 M allocations: 1.825 GiB, 0.72% gc time)
184.279242 seconds (36.97 M allocations: 1.825 GiB, 0.71% gc time)
185.421500 seconds (36.92 M allocations: 1.822 GiB, 0.72% gc time)
182.626996 seconds (36.92 M allocations: 1.822 GiB, 0.74% gc time)
178.672507 seconds (36.92 M allocations: 1.822 GiB, 0.73% gc time)
175.638683 seconds (36.92 M allocations: 1.822 GiB, 0.73% gc time)
173.673905 seconds (36.92 M allocations: 1.822 GiB, 0.73% gc time)
167.288676 seconds (35.29 M allocations: 1.742 GiB, 0.73% gc time)
171.341500 seconds (35.90 M allocations: 1.773 GiB, 0.73% gc time)
169.682736 seconds (35.90 M allocations: 1.773 GiB, 0.74% gc time

146.904167 seconds (29.94 M allocations: 1.479 GiB, 0.73% gc time)
148.367232 seconds (29.94 M allocations: 1.479 GiB, 0.73% gc time)
144.114934 seconds (29.94 M allocations: 1.479 GiB, 0.84% gc time)
144.308501 seconds (29.94 M allocations: 1.479 GiB, 0.85% gc time)
146.839155 seconds (30.48 M allocations: 1.505 GiB, 0.86% gc time)
148.296678 seconds (30.48 M allocations: 1.505 GiB, 0.87% gc time)
147.545324 seconds (30.48 M allocations: 1.505 GiB, 0.86% gc time)
149.580284 seconds (30.48 M allocations: 1.505 GiB, 0.85% gc time)
149.426419 seconds (30.48 M allocations: 1.505 GiB, 0.86% gc time)
1, 19 	 2.903291871811559498804827893726809561759735534325557806783417182412432763998448e-20
147.600520 seconds (29.66 M allocations: 1.465 GiB, 0.83% gc time)
148.552743 seconds (29.66 M allocations: 1.465 GiB, 0.85% gc time)
148.653020 seconds (29.66 M allocations: 1.465 GiB, 0.86% gc time)
148.432225 seconds (29.66 M allocations: 1.465 GiB, 0.88% gc time)
145.520823 seconds (29.66 M allocati

LoadError: [91mInterruptException:[39m

In [25]:
N=30
ppp = BF(readcsv("Parrish/prd_$(lpad(N,2,0)).csv")[24,4])
@time OS = NormOrderStatistic{BigFloat}(N);

  9.260612 seconds (6.33 M allocations: 298.942 MiB, 1.11% gc time)


In [26]:
@time abs(expectationproduct(OS, 1, 24) - ppp)
# 1, 24 	 2.35106904164784496240055164573411190974427398 1944696409204483644586341825264859e-18
#            2.35106904164784496240055164573411190974427398 23 98637379292988188551440032879072e-18
#            2.35106904164784496240055164573411190974427398 23 88359729717455033563297514718043e-18


122.889104 seconds (23.77 M allocations: 1.176 GiB, 0.31% gc time)
130.393467 seconds (25.90 M allocations: 1.280 GiB, 0.30% gc time)
135.759706 seconds (28.26 M allocations: 1.396 GiB, 0.32% gc time)
138.940996 seconds (28.80 M allocations: 1.423 GiB, 0.32% gc time)
139.899514 seconds (29.18 M allocations: 1.441 GiB, 0.32% gc time)
141.064383 seconds (29.38 M allocations: 1.451 GiB, 0.33% gc time)
149.314622 seconds (30.75 M allocations: 1.519 GiB, 0.33% gc time)
152.680578 seconds (31.63 M allocations: 1.562 GiB, 0.33% gc time)
156.382870 seconds (32.09 M allocations: 1.585 GiB, 0.33% gc time)
160.867832 seconds (33.03 M allocations: 1.631 GiB, 0.33% gc time)
164.069502 seconds (33.60 M allocations: 1.659 GiB, 0.33% gc time)
163.976337 seconds (33.87 M allocations: 1.672 GiB, 0.34% gc time)
166.968240 seconds (34.21 M allocations: 1.689 GiB, 0.34% gc time)
164.475308 seconds (34.58 M allocations: 1.707 GiB, 0.34% gc time)
165.597666 seconds (34.84 M allocations: 1.720 GiB, 0.34% gc t

122.504946 seconds (24.18 M allocations: 1.197 GiB, 0.65% gc time)
130.796610 seconds (26.36 M allocations: 1.303 GiB, 0.67% gc time)
136.241023 seconds (28.26 M allocations: 1.396 GiB, 0.69% gc time)
141.871282 seconds (29.35 M allocations: 1.450 GiB, 0.69% gc time)
142.943076 seconds (29.65 M allocations: 1.465 GiB, 0.69% gc time)
144.415216 seconds (29.93 M allocations: 1.479 GiB, 0.69% gc time)
150.760625 seconds (31.31 M allocations: 1.547 GiB, 0.68% gc time)
154.318481 seconds (32.22 M allocations: 1.591 GiB, 0.70% gc time)
155.567972 seconds (32.69 M allocations: 1.614 GiB, 0.71% gc time)
157.241501 seconds (33.03 M allocations: 1.631 GiB, 0.69% gc time)
160.094384 seconds (33.60 M allocations: 1.659 GiB, 0.70% gc time)
161.449785 seconds (33.87 M allocations: 1.672 GiB, 0.70% gc time)
165.673550 seconds (34.82 M allocations: 1.720 GiB, 0.71% gc time)
167.002366 seconds (35.20 M allocations: 1.738 GiB, 0.69% gc time)
167.999118 seconds (35.44 M allocations: 1.750 GiB, 0.70% gc t

153.577802 seconds (31.88 M allocations: 1.575 GiB, 0.87% gc time)
156.960672 seconds (32.80 M allocations: 1.620 GiB, 0.88% gc time)
124.895890 seconds (24.61 M allocations: 1.218 GiB, 0.82% gc time)
134.781407 seconds (26.83 M allocations: 1.327 GiB, 0.82% gc time)
141.738374 seconds (29.29 M allocations: 1.446 GiB, 0.87% gc time)
144.289083 seconds (29.86 M allocations: 1.475 GiB, 0.85% gc time)
145.915639 seconds (30.24 M allocations: 1.494 GiB, 0.85% gc time)
147.320124 seconds (30.47 M allocations: 1.505 GiB, 0.87% gc time)
153.120617 seconds (31.88 M allocations: 1.575 GiB, 0.85% gc time)
124.740285 seconds (24.61 M allocations: 1.218 GiB, 0.82% gc time)
135.497411 seconds (27.36 M allocations: 1.353 GiB, 0.84% gc time)
141.071123 seconds (29.29 M allocations: 1.446 GiB, 0.87% gc time)
144.228632 seconds (29.86 M allocations: 1.475 GiB, 0.86% gc time)
146.028239 seconds (30.24 M allocations: 1.494 GiB, 0.85% gc time)
147.234175 seconds (30.47 M allocations: 1.505 GiB, 0.87% gc t

2.351069041647844962400551645734111909744273982388359729717455033563297514718043e-18

-1.576078693669754094209999999999999999999999999999999999999999999999999999999994

2.351069041647844962400551645734111909744273982398637379292988188551440032879072e-18

In [19]:
# test_product(BigFloat, 50, precision=1e-25)

1, 1 	 1.965198442514171625986374808721709053265990642358510675817736406109573884998823e-11
1, 2 	 1.965198442514171625970365928705668674353104861573228612796224649856278693352417e-11
1, 3 	 4.190182614865900436287488836160513318146199915371361486555238572455575724601681e-11
1, 4 	 7.48625045437264322703027411236950346309149474934232246507597123537808457562393e-12
1, 5 	 1.806704743514522986038552655867493972223271137596626917634075640518867997954705e-11
1, 6 	 4.40467785969694652190216151239125072484802032429721507399498280163194484340028e-12
1, 7 	 3.097128760186346571716304114959576881182472591198349603910530013417144592229718e-11
1, 8 	 1.605951790089697515386707851084453954950638237252895350324666843594609688349691e-11
1, 9 	 2.905895488002647595218059138256681706242488074319573225969406700431226877026400e-11
1, 10 	 3.423650715039063467823144525268265984971716747528956684703279547226481364934555e-11
1, 11 	 2.42593049760656966165080129934624380707369899290314706472314586332630296

Progress:   0%|                                         |  ETA: 0:05:20

1.152912737200402052692442642937052203064160841992094767962245568647224837672369e-11
1, 22 	 8.340878127065952109460636422005475451969171693193246957512481677691052487857251e-13
1, 23 	 1.300951049354852661423252021577089807097866614542093446054012385447355858826504e-11
1, 24 	 2.467257389627842973705578036678650275002714056094893089941479620782138421009271e-11
1, 25 	 3.132184639372952239011677106621523603670089535911067102936091295608603098862277e-12


Progress:   3%|█                                        |  ETA: 0:00:19

1, 26 	 3.770152351783026289860554751644621944532985838995128439676415553782110073834034e-11
1, 27 	 2.353928831513278661372296502447850026364564461350083094780779806761337314711957e-11
1, 28 	 9.222841241106721449429658343719566878223587343070403848221920767726495301203869e-13
1, 29 	 3.097079179183600321395935350596421780818315528581873871038487521591748269623036e-10
1, 30 	 

...

Progress:  98%|████████████████████████████████████████ |  ETA: 0:00:00

2.019999111653822708014406189587533620136116131500191681580980513837378466295154e-06
24, 25 	 2.000003901548988090411333817103108121683161748833418055059181638261021666062326e-05
24, 26 	 3.393573944632701257667307193169913393979375096634005329746188527216251018857809e-11
24, 27 	 1.794237803942424796374009016973062833846892358771873776194805541490453961992156e-11
25, 25 	 

Progress:  99%|█████████████████████████████████████████|  ETA: 0:00:00

3.033010976953512870593757128720955879544332350846324051737018592571851362065472e-11
25, 26 	 4.421985264088490620158482462847327248536242941832078388013292611520246251905289e-12


Progress: 100%|█████████████████████████████████████████| Time: 0:00:28


In [16]:
using StatsFuns

In [18]:
using QuadGK

QuadGK.quadgk(::Type{T}, f, a...; kwargs...) where T<:Real = quadgk(f, a...; kwargs...)[1]
gkord(n) = 250+ceil(Int, sqrt(n))
R = 12

function integrand(x::T, j::Int, r::T) where T
    res =  quadgk(T,
        y -> normcdf(y)^j,
        -r, -x, order=gkord(j)
        )::T
    return res
end

function ψ1(i::Int, j::Int, r::T=T(R)) where T
    @time res = quadgk(T,
        x -> exp(i*normlogcdf(x) + log(integrand(x, j, r))),
        -r,  r, order=gkord(i+j)
        )::T
    return res
end

function ψ2(i::Int, j::Int, r::T=T(R)) where T
    @time res = quadgk(T,
        x -> normcdf(x)^i * integrand(x, j, r),
        -r,  r, order=gkord(i+j)
        )::T
    return res
end

ψ2 (generic function with 2 methods)

In [23]:
@show ψ1(1,12,Float64(R))
@show ψ2(1,12,Float64(R));

  0.037932 seconds (22.14 k allocations: 672.984 KiB)
ψ1(1, 12, Float64(R)) = 0.017234314235855835
  0.037902 seconds (22.14 k allocations: 672.984 KiB)
ψ2(1, 12, Float64(R)) = 0.017234314235855832


In [13]:
truecov = BigFloat("0.1466226179786716492838817")
OS = NormOrderStatistic{BigFloat}(10)
@time bigfloatcov = cov(OS, 2, 3)
OSf = NormOrderStatistic{Float64}(10)
@time float64_cov = cov(OSf, 2, 3)
@show truecov - bigfloatcov
@show truecov - float64_cov;

# truecov - bigfloatcov = 3.879082946149961649439948223560278850059121773922826718121723654550658657729534e-26
# truecov - float64_cov = 6.055469437821437332779169082641601562500000000000000000000000009353658145329218e-16

  0.000051 seconds (101 allocations: 4.852 KiB)
  0.152816 seconds (47.19 k allocations: 1.364 MiB)
truecov - bigfloatcov = 3.879082946149961649439948223560278850059121773922826718121723654550658657729534e-26
truecov - float64_cov = 6.055469437821437332779169082641601562500000000000000000000000009353658145329218e-16


In [12]:
using Base.Test
OS10 = NormOrderStatistic{Float64}(10)
v = [-1.5387527, -1.0013571, -0.6560591, -0.3757647, -0.1226678] # Godwin Table 1
@test all(abs.(expectation(OS10) -[v; -reverse(v)]) .< 1e-7)

OS5 = NormOrderStatistic(5)
@test expectationproduct(OS5, 5,4) ≈ 5*√3/4π + 5*√3/(2π^2)*asin(1/4) # Godwin exact value

using JLD
load

@test all(abs.(cov(OS10) .- V10) .< 2e-12)

Test Passed

In [14]:
maximum(abs.(cov(OS10) .- V10))

1.5132339825640884e-12

### E(Ordstat) - Approx

In [15]:
blom(n,i) = norminvcdf((i - 3/8)/(n + 1/4))
rahman(n,i) = norminvcdf(i/(n + 1))

struct Blom
    n::Int
end

struct Rahman
    n::Int
end

OrderStatistics.expectation(B::Blom) = [blom(B.n,i) for i in 1:B.n]
OrderStatistics.expectation(R::Rahman) = [rahman(R.n,i) for i in 1:R.n]

In [96]:
# M. Mahibbur Rahman & Z. Govindarajulu,  
# A modification of the test of Shapiro and Wilk for normality  
# *Journal of Applied Statistics* 24:2, 219-236, 1997  
# DOI: [10.1080/02664769723828](http://dx.doi.org/10.1080/02664769723828)

function asinvV(N, M)
    pdfM = normpdf.(M)
    diagonal = 2*pdfM.^2
    ldiagonal = [-pdfM[i]*pdfM[i+1] for i in 1:N-1]
    
    return (N+1)*(N+2)*SymTridiagonal(diagonal, ldiagonal)
end

function HypothesisTests.SWCoeffs(R::Rahman)
    
    M = expectation(R)
    A = asinvV(R.n, M)
    C = sqrt(M'*(A*A)*M)

    Astar = (M'*A)'/C
    return SWCoeffs(R.n, -(Astar[1:div(R.n,2)]))
end

In [97]:
N = 20
@show SWCoeffs(N)
OS = NormOrderStatistic(N)
@show SWCoeffs(OS)
M = expectation(OS)
A = asinvV(20, M)
Astar = (M'*A)'/sqrt(M'*A*A*M)
@show SWCoeffs(20, -Astar[1:div(20,2)]);

SWCoeffs(N) = HypothesisTests.SWCoeffs(20, [0.473371, 0.32174, 0.255663, 0.208297, 0.16864, 0.133584, 0.101474, 0.0712893, 0.0423232, 0.0140351])
SWCoeffs(OS) = HypothesisTests.SWCoeffs(20, [0.473413, 0.321075, 0.256522, 0.208488, 0.168566, 0.133433, 0.10129, 0.0711536, 0.0422246, 0.0140079])
SWCoeffs(20, -(Astar[1:div(20, 2)])) = HypothesisTests.SWCoeffs(20, [0.210154, 0.417991, 0.330135, 0.26578, 0.213674, 0.168519, 0.127651, 0.0895162, 0.0530862, 0.0175953])


In [98]:
function HypothesisTests.SWCoeffs(OS::NormOrderStatistic)
    m = expectation(OS)
    iV = inv(cov(OS))
    A = m'*iV./sqrt(m'*iV*iV*m)
    return SWCoeffs(OS.n, -A[1,1:div(OS.n,2)])
end

In [99]:
N = 4000
@time EOS = NormOrderStatistic(N)
plot(expectation(EOS))

  1.121032 seconds (135.45 k allocations: 65.036 MiB, 2.07% gc time)


In [100]:
plot(abs.(EOS - expectation(Blom(N)))./abs.(EOS), label="Blom")
plot!(abs.(EOS - expectation(Rahman(N)))./abs.(EOS), label="Rahman", yaxis=:log10)

In [114]:
N = 500
# @time ExactA = SWCoeffs(NormOrderStatistic(N));
# 17.653383 seconds (5.70 M allocations: 168.471 MiB, 0.38% gc time)
# 0.021701 seconds (36.80 k allocations: 1.164 MiB)

In [115]:
plot(SWCoeffs(N).A, label="Blom + 0-correlation OrdStats")
plot!(SWCoeffs(Rahman(N)).A, label="Rahman + limit inverse correlation")
# plot!(ExactA.A, label="Exact formulas, BigFloat")

In [164]:
X = [sort(randn(1000)) for _ in 1:500];
X = hcat(X...)';
plot([cor(X[:,1], X[:,i]) for i in 1:100])

In [172]:
i,j = 1, 500
x, y = X[:,i], X[:,j];
@show cor(x,y)
@show b,a = linreg(x,y)
scatter(x,y, opacity=0.2)
plot!(x-> a*x + b)

cor(x, y) = 0.03888149783867629
(b, a) = linreg(x, y) = (0.015517121124812277, 0.004865508654331547)


In [72]:
X = 
plot(sort(randn(100))[1:15])
plot!(sort(randn(100))[1:15])
plot!(sort(randn(100))[1:15])
plot!(sort(randn(100))[1:15])

In [68]:
10^18*big(0.1) - 10^17

5.551115123125782702118158340454101562500000000000000000000000000000000000000000

In [34]:
plot(abs.((SWCoeffs(N).A - ExactA.A)), yaxis=:log10)

### E(Ostat) - MonteCarlo

In [56]:
struct MonteCarlo
    n::Int
    samples::Int
end
MonteCarlo(n) = MonteCarlo(n,10^5)

MonteCarlo

In [57]:
function OrderStatistics.expectation(MC::MonteCarlo)
    X = Array{Float64}(MC.n)
    s = zeros(MC.n)
    @inbounds @simd for i in 1:MC.samples
        for j in 1:MC.n
            X[j] = randn()
        end
        sort!(X)
        s += X
    end
    s = s/MC.samples
    s[1:div(MC.n,2)] -= s[div(MC.n,2)+1, end]
    return [s[1:div(MC.n,2)]/2; -reverse(s[1:div(MC.n,2)]/2)]
end

In [59]:
let N = 100
    p = plot(expectation(Blom(N)), label="Blom")
    plot!(expectation(Rahman(N)), label="Rahman")
    plot!(expectation(MonteCarlo(N)), label="MC")
    # yaxis!(:log10)
    display(p)
end

### E(Ordstat) - Exact

In [62]:
T = Float64
m = NormOrderStatistic{T}(500)
@show @time OrderStatistics.expectation(m, 215)
m = NormOrderStatistic{T}(500)
@show @time OrderStatistics.expectation(m, 215);
#   0.000261 seconds (76 allocations: 10.375 KiB)
# @time(OrdMoment(Float64, 215, 500)) = -0.1788431343357474
#   0.005794 seconds (11.12 k allocations: 478.656 KiB)
# @time(OrdMoment(Interval{Float64}, 215, 500)) = Interval(-0.17884313434799265, -0.17884313429753798)

  0.059977 seconds (27.11 k allocations: 1.650 MiB)
@time(OrderStatistics.expectation(m, 215)) = -0.17884313433574675
  0.058920 seconds (27.11 k allocations: 1.650 MiB)
@time(OrderStatistics.expectation(m, 215)) = -0.17884313433574675


In [66]:
@code_warntype OrderStatistics.expectation(m, 215)

Variables:
  #self# <optimized out>
  OS::OrderStatistics.NormOrderStatistic{Float64}
  i::Int64

Body:
  begin 
      return $(Expr(:invoke, MethodInstance for getindex(::OrderStatistics.NormOrderStatistic{Float64}, ::Int64), :(OrderStatistics.getindex), :(OS), :(i)))
  end::Float64


In [64]:
n = 5000
@time expectation(NormOrderStatistic(n))[2150]
# 1.751394 seconds (258.24 k allocations: 102.525 MiB, 1.19% gc time)
# -0.17662112675900268
# 1.193528 seconds (396.64 k allocations: 105.838 MiB, 2.80% gc time)
# 1.230779 seconds (354.65 k allocations: 103.868 MiB, 1.39% gc time)

  0.908125 seconds (354.65 k allocations: 103.868 MiB, 3.44% gc time)


-0.1766211267590179

### Comparison

In [34]:
N = 5000
p = plot()
@time mEx = expectation(NormOrderStatistic(N))

for method in [Blom(N), Rahman(N)]
    @time m = expectation(method)
    plot!(abs.(mEx-m), label=method)
end
yaxis!(:log10)
p
#   1.234846 seconds (258.23 k allocations: 102.525 MiB, 1.02% gc time)
#   0.000105 seconds (2 allocations: 19.641 KiB)
#   0.000126 seconds (2 allocations: 19.641 KiB)
#   0.000124 seconds (2 allocations: 19.641 KiB)

  1.150879 seconds (354.65 k allocations: 103.868 MiB, 1.71% gc time)
  0.000175 seconds (2 allocations: 39.141 KiB)
  0.000175 seconds (2 allocations: 39.141 KiB)


In [37]:
m = NormOrderStatistic{IntervalArithmetic.Interval{Float64}}(N)
@time mEX = expectation(m)

 22.960023 seconds (40.59 M allocations: 1.796 GiB, 2.16% gc time)


5000-element Array{IntervalArithmetic.Interval{Float64},1}:
 [-3.67756, -3.67755]
 [-3.42105, -3.42104]
 [-3.28469, -3.28468]
 [-3.19057, -3.19056]
 [-3.11822, -3.11821]
 [-3.0592, -3.05919] 
 [-3.00921, -3.0092] 
 [-2.96577, -2.96576]
 [-2.92728, -2.92727]
 [-2.8927, -2.89269] 
 [-2.86126, -2.86125]
 [-2.83242, -2.83241]
 [-2.80576, -2.80575]
   ⋮                 
  [2.83241, 2.83242] 
  [2.86125, 2.86126] 
  [2.89269, 2.8927]  
  [2.92727, 2.92728] 
  [2.96576, 2.96577] 
  [3.0092, 3.00921]  
  [3.05919, 3.0592]  
  [3.11821, 3.11822] 
  [3.19056, 3.19057] 
  [3.28468, 3.28469] 
  [3.42104, 3.42105] 
  [3.67755, 3.67756] 

## Varience (correct!)

In [40]:
N = 9
@show N
m = NormOrderStatistic(N)
for i in 1:5
    @show moment(m, i, 2) - moment(m, i, 1)^2 #Varience
end
# OrdMoment(i, N, 2) - OrdMoment(i, N, 1) ^ 2 = 0.3573533263578117
# OrdMoment(i, N, 2) - OrdMoment(i, N, 1) ^ 2 = 0.22569687775856284
# OrdMoment(i, N, 2) - OrdMoment(i, N, 1) ^ 2 = 0.1863826133216484
# OrdMoment(i, N, 2) - OrdMoment(i, N, 1) ^ 2 = 0.17055884541203603
# OrdMoment(i, N, 2) - OrdMoment(i, N, 1) ^ 2 = 0.1661012813587196
N = 10
@show N
m = NormOrderStatistic(N)
for i in 1:5
    @show moment(m, i, 2) - moment(m, i, 1)^2 #Varience
end
# OrdMoment(i, N, 2) - OrdMoment(i, N, 1) ^ 2 = 0.34434382326068835
# OrdMoment(i, N, 2) - OrdMoment(i, N, 1) ^ 2 = 0.21452414298276934
# OrdMoment(i, N, 2) - OrdMoment(i, N, 1) ^ 2 = 0.17500328340301335
# OrdMoment(i, N, 2) - OrdMoment(i, N, 1) ^ 2 = 0.1579389143785767
# OrdMoment(i, N, 2) - OrdMoment(i, N, 1) ^ 2 = 0.15105390390822798

N = 9
moment(m, i, 2) - moment(m, i, 1) ^ 2 = 0.3573533263578117
moment(m, i, 2) - moment(m, i, 1) ^ 2 = 0.22569687775856262
moment(m, i, 2) - moment(m, i, 1) ^ 2 = 0.18638261332164796
moment(m, i, 2) - moment(m, i, 1) ^ 2 = 0.170558845412036
moment(m, i, 2) - moment(m, i, 1) ^ 2 = 0.16610128135871968
N = 10
moment(m, i, 2) - moment(m, i, 1) ^ 2 = 0.344343823260687
moment(m, i, 2) - moment(m, i, 1) ^ 2 = 0.21452414298276912
moment(m, i, 2) - moment(m, i, 1) ^ 2 = 0.1750032834030129
moment(m, i, 2) - moment(m, i, 1) ^ 2 = 0.1579389143785767
moment(m, i, 2) - moment(m, i, 1) ^ 2 = 0.15105390390822795


## Covariance - Exact

In [7]:
function test_all(N=100, ε = eps(Float64))
    MOD = OrderStatistics
    c = 0
    for i in 1:N
        @time for j in 1:N
            if i != j
    #             @show i,j
                v = MOD.α(i,j) - MOD.α(i,j+1) - MOD.α(i+1,j)
                if abs(v) > ε
                    c+=1
                    @show (i,j,v)
                end
                w = MOD.α(i,j) + MOD.α(j,i)
                if abs(w) > ε
                    c+=1
                    @show (i,j,w)
                end

                v = MOD.β(i,j) - MOD.β(i,j+1) - MOD.β(i+1,j)
                if abs(v) > ε
                    c+=1
                    @show (i,j,v)
                end
                w = MOD.β(i,j) - MOD.β(j,i)
                if abs(w) > ε
                    c+=1
                    @show (i,j,w)
                end

                v = MOD.ψ(i,j) - MOD.ψ(j,i)
                if abs(v) > ε
                    c+=1
                    @show (i,j,v)
                end
            end
        end
    end
    return c
end

test_all (generic function with 3 methods)

In [8]:
test_all(100)

LoadError: [91merfcx(8.426975604888928524415319896034829941166176087839361056170337527422070636302799,) has been moved to the package SpecialFunctions.jl.
Run Pkg.add("SpecialFunctions") to install SpecialFunctions on Julia v0.6 and later,
and then run `using SpecialFunctions`.[39m

In [9]:
N = 10
m = NormOrderStatistic(N)
@time OrderStatistics.moment(m, 1, 2) - m[1]*m[1]
@time cov(m,1,1)
#   0.000460 seconds (274 allocations: 8.500 KiB)
#   0.000176 seconds (78 allocations: 2.672 KiB)


  0.010305 seconds (125 allocations: 4.734 KiB)
  0.000168 seconds (92 allocations: 2.531 KiB)


0.344343823260683

In [10]:
using ProgressMeter

In [3]:
N = 10
m = NormOrderStatistic(N)
@time cov(m)

LoadError: [91mUndefVarError: Os not defined[39m

In [184]:
function covarianceM(m)
    n = length(m)
    V = zeros(n,n)
    @showprogress for j in 1:n
        for i in j:n
            V[i,j] = OrdProduct(i,j,n) - m[i]*m[j]
        end
        V[j, j+1:end] .= V[j+1:end, j]
    end
    return V
end

covarianceM (generic function with 1 method)

In [ ]:
using JLD
V10 = load("./V10.jld", "V")
N = 10
m = OrdStats(:Exact, N)
@time V = covarianceM([m; -reverse(m)])
all(V .≈ V10)

## Try @cached 

In [99]:
macro dump(ex)
    s = ex.args[1].args[1].args[2:end]
    println(s, typeof(s))
    args = ex.args[1].args[1].args[2:end]
    argtypes=[]
    for a in args
        a.head == :(::) || error("You need to provide types for all arguments!")
        push!(argtypes, a.args[2])
    end
    
    println(Dict{Tuple{eval.(argtypes)...}, Float64}())
    println(dump(ex))
end

for T in [Float64, BigFloat]
    @eval begin
        @dump function ffff(i::Int, j::Int, r::$T)::$T
            return r*(i+j)
        end
    end
end

Any[:(i::Int), :(j::Int), :(r::Float64)]Array{Any,1}
Dict{Tuple{Int64,Int64,Float64},Float64}()
Expr
  head: Symbol function
  args: Array{Any}((2,))
    1: Expr
      head: Symbol ::
      args: Array{Any}((2,))
        1: Expr
          head: Symbol call
          args: Array{Any}((4,))
            1: Symbol ffff
            2: Expr
              head: Symbol ::
              args: Array{Any}((2,))
                1: Symbol i
                2: Symbol Int
              typ: Any
            3: Expr
              head: Symbol ::
              args: Array{Any}((2,))
                1: Symbol j
                2: Symbol Int
              typ: Any
            4: Expr
              head: Symbol ::
              args: Array{Any}((2,))
                1: Symbol r
                2: Float64 <: AbstractFloat
              typ: Any
          typ: Any
        2: Float64 <: AbstractFloat
      typ: Any
    2: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: Expr
          hea

In [ ]:
function argnamestypes(args)
    argnames = Symbol[]
    argtypes = Symbol[]
    for a in args
        a.head == :(::) || error("You need to provide types for all arguments!")
        push!(argnames, a.args[1])
        push!(argtypes, a.args[2])
    end
    return argnames, argtypes
end


macro cached(ex)
    if ex.head != :function
        error("@cached can be applied only to functions")
    end
    if ex.args[1].head != :(::)
        error("You need to provide return type!")
    else
        returntype = eval(ex.args[1].args[2])
    end
    @assert ex.args[1].args[1] == :call

    fname = ex.args[1].args[1].args[1]
    args = ex.args[1].args[1].args[2:end])
    argnames, argtypes = argnamestypes(args)

    if !(haskey(_cache, fname))
        _cache[fname] = Dict{Type, Dict}()
    end

    if !(haskey(_cache[fname][returntype]))
        _cache[fname][returntype] = Dict{Type, Dict}()


    end
end

### MC approximation of perfect SWCoeffs

In [146]:
function approximate_A(N, samps)
    s = [sort(randn(N)) for i in 1:samps]
    m = sum(s[i][1:N] for i in 1:samps)/samps
    ss = vcat(s...)
    Vinv = inv(cov(ss, ss))
    A = (-m'*Vinv/sqrt(m'*Vinv*Vinv*m))'
    return ((A - reverse(A))/2)[1:div(N,2)]
end    

approximate_A (generic function with 1 method)

In [149]:
A_approx = approximate_A(10,1000000)
@show A_approx
A_Roystn = SWCoeffs(10).A
@show A_Roystn
A_Rahman = SWCoeffs(10, Val{:Rahman}).A
@show A_Rahman;

A_approx = [0.546836, 0.356067, 0.233275, 0.133651, 0.0436704]
A_Roystn = [0.573715, 0.32897, 0.214349, 0.122791, 0.0400887]
A_Rahman = [0.601031, 0.297509, 0.192125, 0.109792, 0.0358307]


In [87]:
APPRX = [approximate_A(i, 100000) for i in [4,5,6,7,8,9,10,12,15,20,25,30,40,50,75,100,125,150,200,250,350,500,750,1000]]

24-element Array{RowVector{Float64,Array{Float64,1}},1}:
 [0.677196 0.195124 -0.197223 -0.681495]    
 [0.650918 0.2748 … -0.277836 -0.650843]    
 [0.625296 0.316494 … -0.316072 -0.623913]  
 [0.60144 0.336972 … -0.336951 -0.601611]   
 [0.580063 0.346565 … -0.349031 -0.582343]  
 [0.562665 0.353302 … -0.353849 -0.563577]  
 [0.547215 0.355395 … -0.35614 -0.547105]   
 [0.519728 0.355833 … -0.355187 -0.518584]  
 [0.485917 0.349078 … -0.349089 -0.485487]  
 [0.443992 0.335007 … -0.334678 -0.443872]  
 [0.413467 0.320854 … -0.320082 -0.413038]  
 [0.388982 0.307867 … -0.307706 -0.38925]   
 [0.353001 0.286263 … -0.28671 -0.353313]   
 [0.326724 0.269218 … -0.269162 -0.326549]  
 [0.282378 0.238665 … -0.238738 -0.282599]  
 [0.254285 0.217792 … -0.21782 -0.254301]   
 [0.23415 0.202333 … -0.202256 -0.233978]   
 [0.218548 0.190138 … -0.190109 -0.218451]  
 [0.195634 0.171948 … -0.171877 -0.195539]  
 [0.179341 0.158626 … -0.158749 -0.179479]  
 [0.157092 0.140231 … -0.14033 -0.157157]  

## Tests

In [36]:
using Base.Test

In [38]:
# **Worked Example** (Section 4) from
# PATRICK ROYSTON Approximating the Shapiro-Wilk W-test for non-normality  
# *Statistics and Computing* (1992) **2**, 117-119  

X = [48.4, 49.0, 59.5, 59.6, 60.7, 88.8, 98.2, 109.4, 169.1, 227.1]
swc = SWCoeffs(length(X))
@test all(abs.(swc.A .- [0.5737, 0.3290, 0.2143, 0.1228, 0.0401]) .< 5.0e-5)
W = HypothesisTests.swstat(X, swc)
@test abs(W - 0.8078) < 2.9e-5
pval = pvalue(W, swc)
@test abs(pval - 0.018) < 4.7e-5

t = ShapiroWilkTest(X)
@test t.W == W
@test pvalue(t) == pval
@test t.N1 == length(X)
@test t.SWc.N == length(X)
@test length(t.SWc) == length(X)

Test Passed

In [39]:
# Values obtained by calling directly `_swilk` fortran subroutine directly.

S = SWCoeffs(10)
a = S.A .- [0.573715, 0.32897, 0.214349, 0.122791, 0.0400887]
@test norm(a,1) < S.N*eps(1.0f0)

S = SWCoeffs(20)
b = S.A .- [0.473371, 0.32174, 0.255663, 0.208297, 0.16864, 0.133584, 0.101474, 0.0712893, 0.0423232, 0.0140351]
@test norm(b,1) < S.N*eps(1.0f0)

Test Passed

In [41]:
using Plots
plotly()

Plots.PlotlyBackend()

In [54]:
N = 5000
A = SWCoeffs(N)
temp = [ShapiroWilkTest(sort(randn(N)), A) for _ in 1:10^6];

In [55]:
histogram([a.W for a in temp], normalize=true)

In [53]:
histogram(pvalue.(temp), normalize=true)

In [ ]:
count(p-> p<0.05, pvals)/length(pvals)

In [ ]:
Ws = [t.W for t in temp];

In [ ]:
histogram(Ws, normalize=true)

In [15]:
X = [4.2,4.9, 5.2, 5.3,6.7,6.7,7.2,7.5, 8.1,8.6, 8.8, 9.3, 9.5,10.3,10.8, 
    11.1,12.2, 12.5, 13.3, 15.1,15.3, 16.1,19.0,19]
ShapiroWilkTest(X)

Shapiro-Wilk normality test
---------------------------
Population details:
    parameter of interest:   Squared correlation of data and SWCoeffes (W)
    value under h_0:         1.0
    point estimate:          0.9439973927768645

Test summary:
    outcome with 95% confidence: fail to reject h_0
    one-sided p-value:           0.2001

Details:
    number of observations:         24
    censored ratio:                 0.0
    W-statistic:                    0.9439973927768645


In [14]:
w, pval, _ , A = swilkfort(X)

(0.9439973975941067, 0.20014069974422455, 0, [0.447513, 0.313024, 0.255022, 0.214319, 0.180601, 0.151129, 0.124462, 0.0997312, 0.0763592, 0.0539304, 0.0321228, 0.0106693])

## Calling Fortran directly

In [10]:
run(`gfortran -shared -fPIC swilk.f -o swilk.so`)
run(`gfortran -fdefault-integer-8 -fdefault-real-8 -shared -fPIC swilk.f -o swilk64.so`)

In [11]:
for (lib, I, F) in (("./swilk64.so", Int64, Float64),
                    ("./swilk.so"  , Int32, Float32))
    @eval begin
        function swilkfort!(X::AbstractVector{$F}, A::AbstractVector{$F}, computeA=true)

            w, pval = Ref{$F}(0.0), Ref{$F}(0.0)
            ifault = Ref{$I}(0)

            ccall((:swilk_, $lib),
                Void,
                (
                Ref{Bool},  # INIT if false compute SWCoeffs in A, else use A
                Ref{$F},    # X    sample
                Ref{$I},    # N    samples length
                Ref{$I},    # N1   (upper) uncensored data length
                Ref{$I},    # N2   length of A
                Ref{$F},    # A    A
                Ref{$F},    # W    W-statistic
                Ref{$F},    # PW   p-value
                Ref{$I},    # IFAULT error code (see swilk.f for meaning)
                ),
                !computeA, X, length(X), length(X), length(A), A, w, pval, ifault)
            return (w[], pval[], ifault[], A)
        end
        swilkfort(X::Vector{$F}) = swilkfort!(X, zeros($F, div(length(X),2)))
    end
end

In [12]:
srand(1)
K = 5000
X = sort(randn(Float32, K))
A = zeros(Float32,div(K,2))
@btime swilkfort(X)
@btime swilkfort!(X, A)
@btime swilkfort!(X, A, false)

  195.356 μs (11 allocations: 10.14 KiB)
  194.704 μs (10 allocations: 208 bytes)
  74.687 μs (10 allocations: 208 bytes)


(0.9994338f0, 0.13073847f0, 0, Float32[0.0549106, 0.0487911, 0.046346, 0.045041, 0.0440341, 0.0432105, 0.0425115, 0.0419032, 0.0413637, 0.0408784  …  6.73575f-5, 6.02677f-5, 5.31769f-5, 4.60872f-5, 3.89965f-5, 3.19057f-5, 2.4816f-5, 1.77253f-5, 1.06356f-5, 3.54484f-6])

In [13]:
srand(1);
K = 5000
X = sort(randn(K));
A = zeros(div(K,2));
@btime swilkfort(X);
@btime swilkfort!(X, A);
@btime swilkfort!(X, A, false)

  204.964 μs (12 allocations: 19.86 KiB)
  203.348 μs (10 allocations: 224 bytes)
  75.437 μs (10 allocations: 224 bytes)


(0.999434337768947, 0.13126509739103806, 0, [0.0549106, 0.0487911, 0.046346, 0.045041, 0.0440341, 0.0432104, 0.0425115, 0.0419032, 0.0413637, 0.0408784  …  6.73577e-5, 6.02674e-5, 5.31771e-5, 4.60868e-5, 3.89965e-5, 3.19062e-5, 2.48159e-5, 1.77257e-5, 1.06354e-5, 3.54513e-6])

In [14]:
srand(1)
K = 5000
X = sort(randn(K))
@btime ShapiroWilkTest(X)
swc = SWCoeffs(K)
@btime SWCoeffs(K)
@btime ShapiroWilkTest(X, swc);

  110.186 μs (16 allocations: 117.81 KiB)
  51.077 μs (5 allocations: 39.31 KiB)
  62.312 μs (11 allocations: 78.50 KiB)
